In [29]:
from openai import OpenAI
from tavily import TavilyClient

import os
import json
import re

In [6]:
openai.api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

In [10]:
client = openai
tavily = TavilyClient(api_key=tavily_api_key)

In [17]:

# For basic search:
response = tavily.search(query="Did India win day before's T20 against Afghanistan?", max_results=1)
# For advanced search:
#response = tavily.search(query="Should I invest in Apple in 2024?", search_depth="advanced")
# Get the search results as context to pass an LLM:
context = [{"url": obj["url"], "content": obj["content"]} for obj in response["results"]]

In [18]:
context

[{'url': 'https://www.bbc.co.uk/sport/cricket/articles/cv22y4p21gwo',
  'content': 'Watch the best shots as Suryakumar Yadav hits a sublime 53 off 28 balls to lead India to a comprehensive 47-run victory over Afghanistan in the T20 World Cup Super 8s in Barbados.'}]

In [27]:
context_str = json.dumps({ "context": context })
context_str

'{"context": [{"url": "https://www.bbc.co.uk/sport/cricket/articles/cv22y4p21gwo", "content": "Watch the best shots as Suryakumar Yadav hits a sublime 53 off 28 balls to lead India to a comprehensive 47-run victory over Afghanistan in the T20 World Cup Super 8s in Barbados."}]}'

### Setup agent

In [37]:
class Agent:
    def __init__(self, system="", model="gpt-4o"):
        self.system = system
        self.model = model
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model=self.model,
            temperature=0,
            messages=self.messages
        )
        return completion.choices[0].message.content

In [38]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

web_search:
e.g. web_search: what was the score in yesterday's match
returns web search result for the query "what was the score in yesterday's match"

Example session:

Question: Who won the T20 India match yesterday and by what difference?
Thought: I should do a web search to find out who won the T20 match yesterday involving India.
Action: web_search: what was the score in yesterday's T20 match involving India
PAUSE

You will be called again with this:

Observation: { "context" : [{ 'url': 'www.example.com/cricketscore/match123', 'content': 'India 156/3 (20), Afghanistan 134 (20)' }] }

You then continue the loop:

Thought: I now need to calculate the difference in scores: 156 and 134
Action: calculate: 156 - 134
PAUSE

You will be called again with this:

Observation: 22

You then output

Answer: India won the match by 22 runs. [source: 'www.example.com/cricketscore/match123']
""".strip()

In [39]:
def calculate(what):
    return eval(what)

def web_search(query):
    response = tavily.search(query=query, max_results=3)
    context = [{"url": obj["url"], "content": obj["content"]} for obj in response["results"]]
    return json.dumps({ "context": context })

known_actions = {
    "calculate": calculate,
    "web_search": web_search
}

In [40]:
action_re = re.compile('^Action: (\w+): (.*)$')

def query(question, max_turns=5, model="gpt-4o"):
    i = 0
    bot = Agent(prompt, model)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [42]:
question = """Who won the last T20 match involving India in the ongoing 2024 world cup, by what margin"""
query(question)

Thought: I need to find out the result of the last T20 match involving India in the ongoing 2024 World Cup.
Action: web_search: result of the last T20 match involving India in the ongoing 2024 World Cup
PAUSE
 -- running web_search result of the last T20 match involving India in the ongoing 2024 World Cup
Observation: {"context": [{"url": "https://sports.ndtv.com/t20-world-cup-2024/ind-vs-pak-live-cricket-score-t20-world-cup-2024-match-19th-india-vs-pakistan-live-scorecard-5851956", "content": "India vs Pakistan Highlights, T20 World Cup 2024: Jasprit Bumrah bowled a terrific spell as India edged past Pakistan by 6 runs at Nassau County International Cricket Stadium in New York. Chasing ..."}, {"url": "https://www.bcci.tv/video/5562707/india-vs-pakistan--icc-mens-t20-world-cup--match-highlights", "content": "India vs Pakistan | ICC Men's T20 World Cup 2024 | Match Highlights. 10 Jun, 2024 06:47 mins 142.5 k. Visit BCCI the official bcci.tv website for minute-to-minute LIVE updates."}, 